This notebook can be used to load raw ultraleap data, 
save cleaned dataframes for each block, 
and generate dataframes of distances for further feature extraction

Import public packages and functions

In [141]:
import os
import importlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from itertools import compress


import openpyxl
from datetime import datetime
import math
import statistics as stat
import json

In [142]:
def get_repo_path_in_notebook():
    """
    Finds path of repo from Notebook.
    Start running this once to correctly find
    other modules/functions
    """
    path = os.getcwd()
    repo_name = 'ultraleap_analysis'

    while path[-len(repo_name):] != 'ultraleap_analysis':

        path = os.path.dirname(path)

    return path


In [143]:
repo_path = get_repo_path_in_notebook()
code_path = os.path.join(repo_path, 'code')
os.chdir(code_path)

Import own functions

In [144]:
import import_data.import_and_convert_data as import_dat
import import_data.find_paths as find_paths
import import_data.preprocessing_meta_info as meta_info
import sig_processing.segment_tasks as tasks
import movement_calc.helpfunctions as hp
from feature_extraction import get_features as get_feat

### Loading blocks for feature extraction

Reloading own functions

In [160]:
importlib.reload(import_dat)
importlib.reload(tasks)
importlib.reload(find_paths)
importlib.reload(meta_info)
importlib.reload(hp)
importlib.reload(get_feat)

<module 'feature_extraction.get_features' from '/Users/arianm/Documents/GitHub/ultraleap_analysis/code/feature_extraction/get_features.py'>

Define variables of interest

In [163]:
folder = 'patientdata'
ls_cond = ['m1', 'm0s0', 'm0s1', 'm1s0', 'm1s1']
ls_cam = ['dt', 'vr',  'st']
ls_task = ['ft', 'oc']
ls_side = ['left', 'right']
subs = find_paths.find_available_subs(folder)

In [164]:
for sub in subs:
    for cond in ls_cond:
        for task in ls_task:
            try:
                files = os.listdir(os.path.join(repo_path, 'data','raw_blocks', sub, task, cond, 'dist'))

            except FileNotFoundError:
                continue 
            
            for file in files:

                if file == '.DS_Store':
                    continue

                # Load blocks from patients' blocks dir
                block = pd.read_csv(os.path.join(
                    repo_path, 'data', 'raw_blocks', sub, task, cond, 'dist', file), index_col=0)

                block_features = get_feat.features_across_block(block, 'ft')
                
                feat_path = os.path.join(repo_path, 'data', 'features', sub, task, cond, 'feat_dict')
                if not os.path.exists(feat_path):
                        os.makedirs(feat_path)

                hp.saveasjson(feat_path, f'{file}', block_features)

/Users/arianm/Documents/GitHub/ultraleap_analysis/code/feature_extraction/get_features.py:24: RuntimeWarning: Mean of empty slice
  block_features['mean_max_amp'].append(np.nanmean(within_features['max_amp']))
/Users/arianm/anaconda3/envs/ultraleap/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1670: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/arianm/Documents/GitHub/ultraleap_analysis/code/feature_extraction/get_features.py:26: RuntimeWarning: Mean of empty slice
  block_features['coef_var_max_amp'].append(np.nanstd(within_features['max_amp'])/np.nanmean(within_features['max_amp']))
/Users/arianm/Documents/GitHub/ultraleap_analysis/code/feature_extraction/get_features.py:30: RuntimeWarning: Mean of empty slice
  block_features['mean_max_vel'].append(np.nanmean(within_features['max_vel']))
/Users/arianm/Documents/GitHub/ultraleap_analysis/code/feature_extraction/get_features.py:32: RuntimeWarning: Mean o